In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

# Подключал гугл диск, так как учил сеть на колабе,
# и на гугл диске хранил всеа лучшей модели, дообучая её

# можно закомментировать, если не работаем с диском

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from google.colab import files
files.upload()

# Подгружал файл kaggle.json во временное хранилище, чтобы скачать датасет с кагла
# Можно закомментировать, если файл есть в директории

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pan4sf","key":"ea0d0e880217e023e52ea93838a8a624"}'}

In [0]:
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets list --mine
!kaggle datasets download -d pan4sf/made-cv-hw1
!unzip \*.zip
!rm made-cv-hw1.zip
!kaggle datasets download -d pan4sf/test-points
!unzip test-points.zip
!rm test-points.zip

# Скачивание датасета и распаковка, test_points.csv распаковывается в корень
# с ноутбуком

# Можно закомментировать, если датасет уже имеется, надо только test_points.csv
# продублировать в папку с ноутбуком

In [0]:
PATH_TO_DATA = 'data/'
yd = 'test_points.csv'
import os

import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils import data

import matplotlib.pyplot as plt
import pickle
import cv2

import sys
from argparse import ArgumentParser

import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import tqdm
from torch.nn import functional as fnn
from torchvision import transforms

np.random.seed(1234)
torch.manual_seed(1234)

TRAIN_SIZE = 0.995
NUM_PTS = 971
CROP_SIZE = 128 # 128
SUBMISSION_HEADER = "file_name,Point_M0_X,Point_M0_Y,Point_M1_X,Point_M1_Y,Point_M2_X,Point_M2_Y,Point_M3_X,Point_M3_Y,Point_M4_X,Point_M4_Y,Point_M5_X,Point_M5_Y,Point_M6_X,Point_M6_Y,Point_M7_X,Point_M7_Y,Point_M8_X,Point_M8_Y,Point_M9_X,Point_M9_Y,Point_M10_X,Point_M10_Y,Point_M11_X,Point_M11_Y,Point_M12_X,Point_M12_Y,Point_M13_X,Point_M13_Y,Point_M14_X,Point_M14_Y,Point_M15_X,Point_M15_Y,Point_M16_X,Point_M16_Y,Point_M17_X,Point_M17_Y,Point_M18_X,Point_M18_Y,Point_M19_X,Point_M19_Y,Point_M20_X,Point_M20_Y,Point_M21_X,Point_M21_Y,Point_M22_X,Point_M22_Y,Point_M23_X,Point_M23_Y,Point_M24_X,Point_M24_Y,Point_M25_X,Point_M25_Y,Point_M26_X,Point_M26_Y,Point_M27_X,Point_M27_Y,Point_M28_X,Point_M28_Y,Point_M29_X,Point_M29_Y\n"

In [0]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
import albumentations as albu

mapping = pd.read_csv('data/train/landmarks.csv', nrows=40000, delimiter = '\t')
cols = [el for el in mapping.columns if el != 'file_name']
heads = ','.join(mapping.columns.values)

wf = open('dop1.csv', 'w')
wf.write(heads+'\n')

def create_transformer(transformations):
    return albu.Compose(transformations, p=1, keypoint_params=albu.KeypointParams(format='xy'))(image=im1, keypoints=lm1)

for i, row in tqdm.tqdm_notebook(mapping.iterrows()):
    file_name = row[0]
    im1 = cv2.imread('data/train/images/'+file_name)
    lm1 = np.array(row[cols].values).reshape(971,2)
    try:
      transformed = create_transformer([albu.ShiftScaleRotate(rotate_limit=20, p=1)])
      keypoints_ = transformed['keypoints']
      im_ = transformed['image']
      if len(keypoints_) != 971:
        continue
      k_ = list(np.array(keypoints_).reshape(2*971))
      cv2.imwrite('data/train/images/'+'R_'+file_name, im_)
      wf.write('R_'+file_name + ',' + ','.join(map(str, k_)) + '\n')
    except:
      continue
wf.close()
# pd.read_csv('dop1.csv').tail()
del mapping
import gc
gc.collect()

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-xa4gvlrf
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-xa4gvlrf
     |████████████████████████████████| 634kB 2.6MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=65055 sha256=8fbe9a197e880dff1eafbe60c600a0e10258f2b4915768b28c10a11e054d6564
  Stored in directory: /tmp/pip-ephem-wheel-cache-aplhlbgb/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=0071b38294df8b13bcda58aa64484c638a0c4b6ed425669b46e92585ac90ebc1
  Stored in directory: /tmp/pip-ephem-wheel-cache-aplhlbgb/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: album

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


4

In [0]:
class ScaleMinSideToSize(object):
    def __init__(self, size=(CROP_SIZE, CROP_SIZE), elem_name='image'):
        self.size = torch.tensor(size, dtype=torch.float)
        self.elem_name = elem_name

    def __call__(self, sample):
        h, w, _ = sample[self.elem_name].shape
        if h > w:
            f = self.size[0] / w
        else:
            f = self.size[1] / h

        sample[self.elem_name] = cv2.resize(sample[self.elem_name], None, fx=f, fy=f, interpolation=cv2.INTER_AREA)
        sample["scale_coef"] = f

        if 'landmarks' in sample:
            landmarks = sample['landmarks'].reshape(-1, 2).float()
            landmarks = landmarks * f
            sample['landmarks'] = landmarks.reshape(-1)

        return sample

In [0]:
class CropCenter(object):
    def __init__(self, size=128, elem_name='image'):
        self.size = size
        self.elem_name = elem_name

    def __call__(self, sample):
        img = sample[self.elem_name]
        h, w, _ = img.shape
        margin_h = (h - self.size) // 2
        margin_w = (w - self.size) // 2
        sample[self.elem_name] = img[margin_h:margin_h + self.size, margin_w:margin_w + self.size]
        sample["crop_margin_x"] = margin_w
        sample["crop_margin_y"] = margin_h

        if 'landmarks' in sample:
            landmarks = sample['landmarks'].reshape(-1, 2)
            landmarks -= torch.tensor((margin_w, margin_h), dtype=landmarks.dtype)[None, :]
            sample['landmarks'] = landmarks.reshape(-1)

        return sample

In [0]:
class TransformByKeys(object):
    def __init__(self, transform, names):
        self.transform = transform
        self.names = set(names)

    def __call__(self, sample):
        for name in self.names:
            if name in sample:
                sample[name] = self.transform(sample[name])

        return sample

In [0]:
# class ThousandLandmarksDataset(data.Dataset):
#     # def __init__(self, root, transforms, a, b, split="train"):
#     def __init__(self, root, transforms, split="train"):
#         super(ThousandLandmarksDataset, self).__init__()
#         self.root = root
#         landmark_file_name = os.path.join(root, 'landmarks.csv') if split is not "test" \
#             else os.path.join(root, "test_points.csv")
#         images_root = os.path.join(root, "images")
#         df = pd.read_csv(landmark_file_name, delimiter='\t') if split != 'test' else pd.read_csv(yd, delimiter='\t')

#         if split is not "test":
#           dop1 = pd.read_csv('dop1.csv')
#           df = pd.concat([dop1, df], axis=0, ignore_index=True)

#         self.landmarks = []
#         self.image_names = []

#         split_idxs = {"train": range(0, int(TRAIN_SIZE * len(df))),
#                       "val": range(int(TRAIN_SIZE * len(df)), len(df)),
#                       "test": range(len(df))}

#         # split_idxs = {"train": a,
#         #               "val": b,
#         #               "test": range(len(df))}

#         idxs = split_idxs[split]

#         if split in ("train", "val"):
#             for row in tqdm.tqdm_notebook(df._values[idxs]):
#                 self.image_names.append(os.path.join(images_root, row[0]))
#                 self.landmarks.append(row[1:].astype('int32').reshape((len(row) // 2, 2)))
#             self.landmarks = torch.as_tensor(self.landmarks)
#         elif split == 'test':
#             for row in tqdm.tqdm_notebook(df._values[idxs]):
#                 self.image_names.append(os.path.join(images_root, row[0]))
#             self.landmarks = None
#         else:
#             raise NotImplementedError(split)

#         self.transforms = transforms

#     def __getitem__(self, idx):
#         sample = {}
#         if self.landmarks is not None:
#             landmarks = self.landmarks[idx]
#             sample["landmarks"] = landmarks

#         image = cv2.imread(self.image_names[idx])
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         sample["image"] = image

#         if self.transforms is not None:
#             sample = self.transforms(sample)

#         return sample

#     def __len__(self):
#         return len(self.image_names)

In [0]:
class ThousandLandmarksDataset(data.Dataset):
    def __init__(self, root, transforms, index_list, split="train"):
        super(ThousandLandmarksDataset, self).__init__()
        self.root = root  # !!!!!!!!!!!!!!!!!!!!
        landmark_file_name = os.path.join(root, 'landmarks.csv') if split is not "test" \
            else yd # os.path.join(root, "test_points.csv")
        images_root = os.path.join(root, "images")

        self.image_names = []
        self.landmarks = []

        # with open(landmark_file_name, "rt") as fp:
        #     num_lines = sum(1 for line in fp)
        # num_lines -= 1  # header

        if split in ("train", "val"):
            with open(landmark_file_name, "rt") as fp:
                for i, line in tqdm.tqdm_notebook(enumerate(fp)):
                    if i == 0:
                        continue  # skip header
                    # print(index_list)
                    if i in index_list:
                        elements = line.strip().split("\t")
                        # print(elements)
                        image_name = os.path.join(images_root, elements[0]) # !!!!!!!!!!!!!!!!!!!!
                        self.image_names.append(image_name)

                        if split in ("train", "val"):
                            landmarks = list(map(np.int16, elements[1:])) # !!!!!!!!!!!!!!!!!!!!
                            landmarks = np.array(landmarks, dtype=np.int16).reshape((len(landmarks) // 2, 2))
                            self.landmarks.append(landmarks)
                    # if i == 1000:
                    #   break

        if split == "train":
            with open('dop1.csv', "rt") as fp:
                for i, line in tqdm.tqdm_notebook(enumerate(fp)):
                    if i == 0:
                        continue  # skip header
                    elements = line.strip().split(",")
                    # print(elements[0])
                    # print(elements[1])
                    image_name = os.path.join(images_root, elements[0]) # !!!!!!!!!!!!!!!!!!!!
                    self.image_names.append(image_name)

                    landmarks = list(map(np.int16, elements[1:])) # !!!!!!!!!!!!!!!!!!!!
                    landmarks = np.array(landmarks, dtype=np.int16).reshape((len(landmarks) // 2, 2))
                    self.landmarks.append(landmarks)
                    # if i == 2000:
                    #     break

        if split == "test":
            with open(landmark_file_name, "rt") as fp:
                for i, line in tqdm.tqdm_notebook(enumerate(fp)):
                    if i == 0:
                        continue  # skip header
                    # print(index_list)
                    if i in index_list:
                        elements = line.strip().split("\t")
                        # print(elements)
                        image_name = os.path.join(images_root, elements[0])
                        self.image_names.append(image_name)
                    # if i == 2000:
                    #     break

        if split in ("train", "val"):
            self.landmarks = torch.as_tensor(self.landmarks)
        else:
            self.landmarks = None

        self.transforms = transforms

    def __getitem__(self, idx):
        sample = {}
        if self.landmarks is not None:
            landmarks = self.landmarks[idx]
            sample["landmarks"] = landmarks

        image = cv2.imread(self.image_names[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        sample["image"] = image

        if self.transforms is not None:
            sample = self.transforms(sample)
        #print (sample)
        return sample

    def __len__(self):
        return len(self.image_names)

In [0]:
def restore_landmarks(landmarks, f, margins):
    dx, dy = margins
    landmarks[:, 0] += dx
    landmarks[:, 1] += dy
    landmarks /= f
    return landmarks


def restore_landmarks_batch(landmarks, fs, margins_x, margins_y):
    landmarks[:, :, 0] += margins_x[:, None]
    landmarks[:, :, 1] += margins_y[:, None]
    landmarks /= fs[:, None, None]
    return landmarks


def create_submission(path_to_data, test_predictions, path_to_submission_file):
    test_dir = os.path.join(path_to_data, "test")

    output_file = path_to_submission_file
    wf = open(output_file, 'w')
    wf.write(SUBMISSION_HEADER)

#     mapping_path = os.path.join(test_dir, 'test_points.csv')
#     mapping = pd.read_csv(mapping_path, delimiter='\t')
    mapping = pd.read_csv(yd, delimiter='\t')

    for i, row in mapping.iterrows():
        file_name = row[0]
        point_index_list = np.array(eval(row[1]))
        points_for_image = test_predictions[i]
        needed_points = points_for_image[point_index_list].astype(np.int)
        wf.write(file_name + ',' + ','.join(map(str, needed_points.reshape(2 * len(point_index_list)))) + '\n')

In [0]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def parse_arguments():
    parser = ArgumentParser(__doc__)
    parser.add_argument("--name", "-n", help="Experiment name (for saving checkpoints and submits).",
                        default="baseline")
    parser.add_argument("--data", "-d", help="Path to dir with target images & landmarks.", default=None)
    parser.add_argument("--batch-size", "-b", default=512, type=int)  # 512 is OK for resnet18 finetune @ 6Gb of VRAM
    parser.add_argument("--epochs", "-e", default=1, type=int)
    parser.add_argument("--learning-rate", "-lr", default=1e-3, type=float)
    parser.add_argument("--gpu", action="store_true")
    return parser.parse_args()

In [0]:
def train(model, loader, loss_fn, optimizer, device):
    model.train()
    train_loss = []
    for batch in tqdm.tqdm_notebook(loader, total=len(loader), desc="training..."):
        images = batch["image"].to(device)  # B x 3 x CROP_SIZE x CROP_SIZE
        landmarks = batch["landmarks"]  # B x (2 * NUM_PTS)

        pred_landmarks = model(images).cpu()  # B x (2 * NUM_PTS)
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         scheduler.step()

    return np.mean(train_loss)

def validate(model, loader, loss_fn, device):
    model.eval()
    val_loss = []
    for batch in tqdm.tqdm_notebook(loader, total=len(loader), desc="validation..."):
        images = batch["image"].to(device)
        landmarks = batch["landmarks"]

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        val_loss.append(loss.item())

    return np.mean(val_loss)


def predict(model, loader, device):
    model.eval()
    predictions = np.zeros((len(loader.dataset), NUM_PTS, 2))
    for i, batch in enumerate(tqdm.tqdm_notebook(loader, total=len(loader), desc="test prediction...")):
        images = batch["image"].to(device)

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        pred_landmarks = pred_landmarks.numpy().reshape((len(pred_landmarks), NUM_PTS, 2))  # B x NUM_PTS x 2

        fs = batch["scale_coef"].numpy()  # B
        margins_x = batch["crop_margin_x"].numpy()  # B
        margins_y = batch["crop_margin_y"].numpy()  # B
        prediction = restore_landmarks_batch(pred_landmarks, fs, margins_x, margins_y)  # B x NUM_PTS x 2
        predictions[i * loader.batch_size: (i + 1) * loader.batch_size] = prediction

    return predictions

In [0]:
class Arguments(object):
    def __init__(self, data = None, batch_size = 512, learning_rate = 1e-3, epochs = 1, name = "baseline"):
        self.data = data
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.name = 'bs_'+str(self.batch_size)+'_e_'+str(self.epochs)+'_resnext50_32x4d_change_lr_overfit_40krot_colab' 
        
args = Arguments(data = PATH_TO_DATA, batch_size = 256, learning_rate = 1e-3, epochs = 18)

device = torch.device("cuda: 0") if torch.cuda.is_available() else torch.device("cpu")
print (args.data, args.batch_size, args.learning_rate, args.epochs, args.name, device)

data/ 256 0.001 18 bs_256_e_18_resnext50_32x4d_change_lr_overfit_40krot_colab cuda:0


In [0]:
lendf =393930
# 1. prepare data & models
train_transforms = transforms.Compose([
    ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
    CropCenter(CROP_SIZE),
#     AlbuShiftScaleRotate(),
    
    TransformByKeys(transforms.ToPILImage(), ("image",)),
    # TransformByKeys(transforms.RandomGrayscale(p=0.05), ("image",)),
    
    TransformByKeys(transforms.ToTensor(), ("image",)),
    # TransformByKeys(transforms.RandomErasing(p=0.1), ("image",)),
    TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), ("image",)),
])

print("Reading data...")
train_dataset = ThousandLandmarksDataset(args.data + 'train', train_transforms, range(0,int(0.995*lendf)), split="train")
train_dataloader = data.DataLoader(train_dataset, batch_size=args.batch_size, num_workers=4, pin_memory=True,
                                   shuffle=True, drop_last=True)
print(len(train_dataset))

val_dataset = ThousandLandmarksDataset(args.data + 'train', train_transforms, list(range(int(0.995*lendf),lendf)), split="val")
val_dataloader = data.DataLoader(val_dataset, batch_size=args.batch_size, num_workers=4, pin_memory=True,
                                 shuffle=False, drop_last=False)
print(len(val_dataset))

Reading data...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



431876



1970


In [0]:
print("Creating model...")
model = models.resnext50_32x4d(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2 * NUM_PTS, bias=True)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, amsgrad=True)
loss_fn = fnn.mse_loss

Creating model...


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth


In [0]:
# 2. train & validate

# Если работаем не в колабе и используем имеющиеся веса, то указываем путь
# к файлу с весами. Если обучаем с нуля, то можно закомментировать эти
# три строки
with open(f"/content/gdrive/My Drive/{args.name}_best.pth", "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)

# 2. train & validate
print("Ready for training...")
best_val_loss = np.inf #np.inf # успелось 5 эпох, дошли до трейн лосса 3.0639, еще 13 эпох с нуля и до 1.4059 дошли
# еще 7 эпох с 5 по 12, дошли до 1.2983, - - - - вот так пришлось 3 раза запустить для лучшего скора

# Это уже дальше переобучал, но не использовал для финального сабмита
# еще 16 эпох с 9 по 25, дошли до 1.2214
# еще 10 эпох с 5 по 10 дошли до 1.0991
# еще с 5 по 21, дошли до 1.0366

for epoch in range(args.epochs):
    if epoch in [0,1,2,3,4]:
        optimizer = optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
#         scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
    elif epoch in [5,6,7,8,9,10,11]:
        optimizer = optim.Adam(model.parameters(), lr=1e-3, amsgrad=True)
#         scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
    elif epoch in [12,13,14,15]:
        optimizer = optim.Adam(model.parameters(), lr=1e-4, amsgrad=True)
#         scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
    else :
        optimizer = optim.Adam(model.parameters(), lr=1e-5, amsgrad=True)
#         scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.5}\tval loss: {:5.5}".format(epoch, train_loss, val_loss))
#     print (optimizer)
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         with open(f"{args.name}_best.pth", "wb") as fp:
#             torch.save(model.state_dict(), fp)
    if train_loss < best_val_loss:
        best_val_loss = train_loss
        with open(f"{args.name}_best.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)
        # Если работаем не с колабом, то надо убрать эту строку
        torch.save(model.state_dict(), f"/content/gdrive/My Drive/{args.name}_best.pth")

Ready for training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Epoch # 0:	train loss: 1.6341	val loss: 1.8755



Epoch # 1:	train loss: 1.6535	val loss: 2.0153


KeyboardInterrupt: ignored

In [0]:
# 3. predict
test_dataset = ThousandLandmarksDataset(os.path.join(args.data, 'test'), train_transforms, range(0,100001), split="test")
test_dataloader = data.DataLoader(test_dataset, batch_size=args.batch_size, num_workers=4, pin_memory=True,
                                  shuffle=False, drop_last=False)

# with open(f"{args.name}_best.pth", "rb") as fp:
#     best_state_dict = torch.load(fp, map_location="cpu")
#     model.load_state_dict(best_state_dict)

# Если работаем не с колабом, то 3 строки выше раскомментировать,
# а три строки ниже закомментировать

with open(f"/content/gdrive/My Drive/{args.name}_best.pth", "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)

test_predictions = predict(model, test_dataloader, device)
with open(f"{args.name}_test_predictions.pkl", "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                 "landmarks": test_predictions}, fp)

create_submission(args.data, test_predictions, f"{args.name}_submit.csv")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [0]:
# create_submission(args.data, test_predictions, "1_submit.csv")

In [0]:
# df1 = pd.read_csv('1_submit.csv')
# df2 = df1.copy()ё
# df_out = df1.copy()
# df_out.head()

In [0]:
# for col in df_out.columns:
#   if col != 'file_name':
#     df_out[col] = pd.Series(np.median(np.vstack((df1[col].values,df2[col].values)), axis=0), dtype=int)
# df_out.tail()

In [0]:
# df_out._values[list(range(1,2))+list(range(3,4))]

In [0]:
# iters = [
#          [range(0,315140), range(315140,393925)],
#          [range(78785,393925), range(0,78785)],
#          [list(range(2*78785,393925))+list(range(0,78785)), range(78785,78785*2)],
#          [list(range(3*78785,393925))+list(range(0,78785*2)), range(78785*2,78785*3)],
#          [list(range(4*78785,393925))+list(range(0,78785*3)), range(78785*3,78785*4)]
#          ]
# ind = 1
# for el in iters:
#   print ('step ',ind)
#   # 1. prepare data & models
#   train_transforms = transforms.Compose([
#       ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
#       CropCenter(CROP_SIZE),
#       TransformByKeys(transforms.ToPILImage(), ("image",)),
#       TransformByKeys(transforms.ToTensor(), ("image",)),
#       TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), ("image",)),
#   ])

#   print("Reading data...")
#   train_dataset = ThousandLandmarksDataset(args.data + 'train', train_transforms,el[0],el[1], split="train")
#   train_dataloader = data.DataLoader(train_dataset, batch_size=args.batch_size, num_workers=4, pin_memory=True,
#                                     shuffle=True, drop_last=True)
#   val_dataset = ThousandLandmarksDataset(args.data + 'train', train_transforms,el[0],el[1], split="val")
#   val_dataloader = data.DataLoader(val_dataset, batch_size=args.batch_size, num_workers=4, pin_memory=True,
#                                   shuffle=False, drop_last=False)
  
#   print("Creating model...")
#   model = models.resnet34(pretrained=True)
#   model.fc = nn.Linear(model.fc.in_features, 2 * NUM_PTS, bias=True)
#   model.to(device)
#   optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, amsgrad=True)
#   loss_fn = fnn.mse_loss
  
#   print("Ready for training...")
#   best_val_loss = np.inf
#   for epoch in range(1,11):
#       train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
#       val_loss = validate(model, val_dataloader, loss_fn, device=device)
#       print("Epoch #{:2}:\ttrain loss: {:5.2}\tval loss: {:5.2}".format(epoch, train_loss, val_loss))
#       if val_loss < best_val_loss:
#           best_val_loss = val_loss
#           with open(f"{args.name}_{ind}_best.pth", "wb") as fp:
#               torch.save(model.state_dict(), fp)

#   # 3. predict
#   test_dataset = ThousandLandmarksDataset(os.path.join(args.data, 'test'), train_transforms,el[0],el[1], split="test")
#   test_dataloader = data.DataLoader(test_dataset, batch_size=args.batch_size, num_workers=4, pin_memory=True,
#                                     shuffle=False, drop_last=False)

#   with open(f"{args.name}_{ind}_best.pth", "rb") as fp:
#       best_state_dict = torch.load(fp, map_location="cpu")
#       model.load_state_dict(best_state_dict)

#   test_predictions = predict(model, test_dataloader, device)
#   with open(f"{args.name}_{ind}_test_predictions.pkl", "wb") as fp:
#       pickle.dump({"image_names": test_dataset.image_names,
#                   "landmarks": test_predictions}, fp)
      
#   create_submission(args.data, test_predictions, f"{ind}_submit.csv")
#   ind+=1

In [0]:
# del train_dataset
# del train_dataloader
# del val_dataset
# del val_dataloader
# del model
# del test_dataset
# del test_dataloader

In [0]:
# import gc
# gc.collect()

In [0]:
# df1 = pd.read_csv('1_submit.csv')
# df2 = pd.read_csv('2_submit.csv')
# df3 = pd.read_csv('3_submit.csv')
# df4 = pd.read_csv('4_submit.csv')
# df5 = pd.read_csv('5_submit.csv')
# df_out = df1.copy()

In [0]:
# for col in df_out.columns:
#   if col != 'file_name':
#     df_out[col] = pd.Series(np.mean(np.vstack((df1[col].values,df2[col].values,df3[col].values,df4[col].values,df5[col].values)), axis=0), dtype=int).astype(int)
# df_out.to_csv(f"{args.name}_mean_submit.csv", index=False)
# df_out.tail()

In [0]:
# !pip install -U git+https://github.com/albu/albumentations --no-cache-dir
# import albumentations as albu

In [0]:
# pd.set_option('display.max_columns', 4000)

In [0]:
# mapping = pd.read_csv('data/train/landmarks.csv', nrows=20000, delimiter = '\t')
# cols = [el for el in mapping.columns if el != 'file_name']
# heads = ','.join(mapping.columns.values)

# wf = open('dop1.csv', 'w')
# wf.write(heads+'\n')

# def create_transformer(transformations):
#     return albu.Compose(transformations, p=1, keypoint_params=albu.KeypointParams(format='xy'))(image=im1, keypoints=lm1)

# for i, row in tqdm.tqdm_notebook(mapping.iterrows()):
#     file_name = row[0]
#     im1 = cv2.imread('data/train/images/'+file_name)
#     lm1 = np.array(row[cols].values).reshape(971,2)
#     try:
#       transformed = create_transformer([albu.ShiftScaleRotate(p=1)])
#       keypoints_ = transformed['keypoints']
#       im_ = transformed['image']
#       if len(keypoints_) != 971:
#         continue
#       k_ = list(np.array(keypoints_).reshape(2*971))
#       cv2.imwrite('data/train/images/'+'R_'+file_name, im_)
#       wf.write('R_'+file_name + ',' + ','.join(map(str, k_)) + '\n')
#     except:
#       continue
# wf.close()
# # pd.read_csv('dop1.csv').tail()
# del mapping
# import gc
# gc.collect()

In [0]:
# mapping.head()

In [0]:
# pd.concat([pd.read_csv('dop1.csv'), mapping], axis=0, ignore_index=True)

In [0]:
# from matplotlib import pyplot as plt
# plt.imshow(im_)

In [0]:
# im1 = cv2.imread('data/train/images/R_98d5186671bbcb31a79a16878a57de77.jpg')
# plt.imshow(im1)

In [0]:
# def vis_points(image, points, diameter=1):
#     im = image.copy()

#     for (x, y) in points:
#         cv2.circle(im, (int(x), int(y)), diameter, (0, 255, 0), -1)

#     plt.imshow(im)

# vis_points(im_, keypoints_)